In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from voting_mechanism_design.agents.quorum_badgeholder import QuorumBadgeholder, QuorumBadgeholderPopulation
from voting_mechanism_design.projects.project import Project
from voting_mechanism_design.funds_distribution.threshold_and_aggregate import ThresholdAndAggregate
from voting_mechanism_design.sim import RoundSimulation

In [ ]:
# Round design variables
total_funds = 100
min_vote = 0
max_vote = 16

In [ ]:
# create badgeholder population
n = 10
badgeholder_list = []
for ii in range(n):
    badgeholder_id = ii
    agent = QuorumBadgeholder(
        badgeholder_id=badgeholder_id,
        total_funds=total_funds,
        min_vote=min_vote,  
        max_vote=max_vote,
        # TODO: these should be random variablescdxsz
        laziness=1,
        expertise=1,
        coi_factor=0
    )
    badgeholder_list.append(agent)
badgeholders = QuorumBadgeholderPopulation(badgeholder_list)

In [ ]:
# create projects
N = 50
projects = []
for ii in range(N):
    project_id = ii
    project = Project(
        project_id=project_id,
        true_impact=1,  # TODO: this should be a random variable
        owner_id=None,  # used for COI modeling
    )

In [ ]:
# instantiate a funding model
fund_distribution_model = ThresholdAndAggregate()

In [ ]:
# create and run a simulation
simulation_obj = RoundSimulation(
    badgeholder_population=badgeholders,
    projects=projects,
    funding_design=fund_distribution_model,
)
simulation_obj.run()

In [ ]:
# get metrics